In [1]:
import time
import urllib

from PIL import Image as PIL_Image
from google import genai
from google.genai import types
import matplotlib.pyplot as plt
import mediapy as media

In [2]:
import os

PROJECT_ID = "thematic-land-451915-j3"  
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

In [3]:
def show_video(gcs_uri):
    file_name = gcs_uri.split("/")[-1]
    !gsutil cp {gcs_uri} {file_name}
    media.show_video(media.read_video(file_name), height=500)


def display_images(image) -> None:
    fig, axis = plt.subplots(1, 1, figsize=(12, 6))
    axis.imshow(image)
    axis.set_title("Starting Image")
    axis.axis("off")
    plt.show()

In [4]:
video_model = "veo-2.0-generate-001"

In [5]:
prompt = "a cat reading a book"  # @param {type: 'string'}
aspect_ratio = "16:9"  # @param ["16:9", "9:16"]
output_gcs = "gs://"  # @param {type: 'string'}

operation = client.models.generate_videos(
    model=video_model,
    prompt=prompt,
    config=types.GenerateVideosConfig(
        aspect_ratio=aspect_ratio,
        output_gcs_uri=output_gcs,
        number_of_videos=1,
        duration_seconds=5,
        person_generation="dont_allow",
        enhance_prompt=True,
    ),
)

while not operation.done:
    time.sleep(15)
    operation = client.operations.get(operation)
    print(operation)

if operation.response:
    show_video(operation.result.generated_videos[0].video.uri)

/var/folders/x5/hz3hvkxn58sdsfnlnm0km3r00000gn/T/ipykernel_20435/484537295.py:5: ExperimentalWarning: This method is experimental and may change in future versions.
  operation = client.models.generate_videos(
/var/folders/x5/hz3hvkxn58sdsfnlnm0km3r00000gn/T/ipykernel_20435/484537295.py:20: ExperimentalWarning: This method is experimental and may change in future versions.
  operation = client.operations.get(operation)


name='projects/thematic-land-451915-j3/locations/us-central1/publishers/google/models/veo-2.0-generate-001/operations/8f21df33-69a0-48c2-aa24-ffb1f14432dd' metadata=None done=None error=None response=None result=None
name='projects/thematic-land-451915-j3/locations/us-central1/publishers/google/models/veo-2.0-generate-001/operations/8f21df33-69a0-48c2-aa24-ffb1f14432dd' metadata=None done=True error={'code': 5, 'message': 'Bucket "11755668437916986467" not found for operation OP_INITIATE_RESUMABLE_WRITE'} response=None result=None


In [11]:
import time
from google.cloud import storage
import os

PROJECT_ID = "thematic-land-451915-j3"
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

# Define your prompt and configurations
prompt = "a cat reading a book"
aspect_ratio = "9:16"
output_gcs = "gs://bucket_nextcopy/video/"

# Initiate video generation
operation = client.models.generate_videos(
    model=video_model,
    prompt=prompt,
    config=types.GenerateVideosConfig(
        aspect_ratio=aspect_ratio,
        output_gcs_uri=output_gcs,
        number_of_videos=1,
        duration_seconds=5,
        person_generation="dont_allow",
        enhance_prompt=True,
    ),
)

# Wait for the operation to complete
while not operation.done:
    time.sleep(15)
    operation = client.operations.get(operation)
    print("Waiting for operation to complete...")

# After the operation is complete
if operation.response:
    video_uri = operation.result.generated_videos[0].video.uri
    print(f"Generated video URI: {video_uri}")

    # Extract bucket name and blob name from the URI
    if video_uri.startswith("gs://"):
        path_parts = video_uri[5:].split("/", 1)
        bucket_name = path_parts[0]
        blob_name = path_parts[1] if len(path_parts) > 1 else ""
        
        if blob_name:
            # Get public URL directly (like your image code)
            storage_client = storage.Client(project=PROJECT_ID)
            bucket = storage_client.bucket(bucket_name)
            blob = bucket.blob(blob_name)
            
            public_url = blob.public_url
            print(f"Public URL: {public_url}")
        else:
            print("Invalid blob path in GCS URI")
    else:
        print("Invalid GCS URI format.")
else:
    print("Video generation failed!")

python(14119) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Waiting for operation to complete...
Waiting for operation to complete...
Waiting for operation to complete...
Generated video URI: gs://bucket_nextcopy/video/3609597647696811937/sample_0.mp4
Public URL: https://storage.googleapis.com/bucket_nextcopy/video/3609597647696811937/sample_0.mp4


python(15112) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
